In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Bayesian import BayesianLinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sns.set_theme()
sns.set_context("notebook")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
dtype_dict = {
    'Date': 'str',
    'FarmName_Pseudo': 'str',
    'SE_Number': 'str',
    'Age': 'Int64',
    'BreedName': 'str',
    'DailyYield': 'float',
    'PreviousDailyYield': 'float',
    'DailyYieldChange': 'float',
    'DaysInMilk': 'Int64',
    'YearSeason': 'str',
    'LactationNumber': 'Int64',
    'ExpectedYield': 'float',
    'NormalizedDailyYield': 'float',
    'NormalizedDailyYieldChange': 'float',
    'HeatStress': 'Int64',
    'Temp15Threshold': 'Int64',
    'HW': 'Int64',
    'cum_HW': 'Int64',
    'MeanTemperature': 'float',
    'MeanTHI_adj': 'float',
    'HeatLoad': 'float',
    'CumulativeHeatLoad': 'float',
}

milk_data = pd.read_csv('../Data/MergedData/HeatApproachYieldDataTest.csv', dtype=dtype_dict)
milk_data['Date'] = pd.to_datetime(milk_data['Date'], format='%Y-%m-%d')
milk_data.head(-5)

,Date,FarmName_Pseudo,SE_Number,Age,BreedName,LactationNumber,DaysInMilk,YearSeason,DailyYield,PreviousDailyYield,...,NormalizedDailyYieldChange,Residuals,HeatStress,Temp15Threshold,HW,cum_HW,MeanTemperature,MeanTHI_adj,HeatLoad,CumulativeHeatLoad
0,2022-05-28,a624fb9a,SE-064c0cec-1189,3242,02 SLB,8,3,2022-2,15.22,NaN,...,NaN,1.820438,0,0,0,0,9.912500,50.478673,-10.521327,0.000000
1,2022-05-29,a624fb9a,SE-064c0cec-1189,3243,02 SLB,8,4,2022-2,18.96,15.22,...,0.215311,1.589745,0,0,0,0,10.066667,53.841648,-7.158352,0.000000
2,2022-05-30,a624fb9a,SE-064c0cec-1189,3244,02 SLB,8,5,2022-2,22.64,18.96,...,0.177389,1.894598,0,1,0,0,10.466667,52.935959,-8.064041,0.000000
3,2022-05-31,a624fb9a,SE-064c0cec-1189,3245,02 SLB,8,6,2022-2,26.49,22.64,...,0.163049,2.877443,0,0,0,0,11.183333,52.872112,-8.127888,0.000000
4,2022-06-01,a624fb9a,SE-064c0cec-1189,3246,02 SLB,8,7,2022-3,33.61,26.49,...,0.273358,7.563598,0,1,0,0,12.704167,56.056547,-4.943453,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576066,2024-08-09,f454e660,SE-f454e660-810,1020,99 Korsning/obestämbar ras,1,365,2024-3,37.53,31.36,...,0.168909,1.001508,1,1,0,0,18.650000,62.298109,1.298109,84.512273
576067,2024-08-10,f454e660,SE-f454e660-810,1021,99 Korsning/obestämbar ras,1,365,2024-3,36.48,37.53,...,-0.028745,-0.048492,1,1,0,0,18.308333,54.687773,-6.312227,71.887820
576068,2024-08-11,f454e660,SE-f454e660-810,1022,99 Korsning/obestämbar ras,1,365,2024-3,34.76,36.48,...,-0.047087,-1.768492,1,1,0,0,17.841667,55.974084,-5.025916,61.835988
576069,2024-08-12,f454e660,SE-f454e660-810,1023,99 Korsning/obestämbar ras,1,365,2024-3,38.06,34.76,...,0.090340,1.531508,1,1,0,0,17.516667,60.115007,-0.884993,60.066002


In [9]:
# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['FarmName_Pseudo', 'FarmCumulativeHeatLoadMilkProduction'])

# Set subject type to 'farm'
subject_type = 'farm'

# Specify the farm ID for analysis
farm_id = 'a624fb9a'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Scale the CumulativeHeatLoad feature
scaler = StandardScaler()
train_data['CumulativeHeatLoad'] = scaler.fit_transform(train_data[['CumulativeHeatLoad']])
val_data['CumulativeHeatLoad'] = scaler.transform(val_data[['CumulativeHeatLoad']])

# Define the single feature
features = ['CumulativeHeatLoad']

# Calculate the correlation between CumulativeHeatLoad and NormalizedDailyYield
cumulative_heatload_correlation = train_data['CumulativeHeatLoad'].corr(train_data[target])
print("Correlation between CumulativeHeatLoad and NormalizedDailyYield:", cumulative_heatload_correlation)

# Function to fit and get the posterior mean for CumulativeHeatLoad
def fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation):
    print(f"\nSelected features: {features}")

    # Calculate prior means for the features
    prior_mean_values = [cumulative_heatload_correlation]  # Use correlation as prior mean for CumulativeHeatLoad
    prior_std_values = [train_data[feature].std() for feature in features]
    
    # Define priors
    prior_mean = np.array([normalized_mean] + prior_mean_values).reshape(-1, 1)
    prior_std = np.array([np.sqrt(normalized_variance)] + prior_std_values).reshape(-1, 1)
    prior_cov = np.eye(len(features) + 1) * 0.01  # Smaller value for stronger prior confidence
    beta = 1 / normalized_variance

    # Print prior means and standard deviations
    print(f"\nPrior Mean and Standard Deviation for each feature:\n")
    for feature, mean, std in zip(['Off-set'] + features, prior_mean.flatten(), prior_std.flatten()):
        print(f"{feature}: mean = {mean}, std_dev = {std}")

    # Initialize and fit the model
    model = BayesianLinearRegression(
        dataframe=train_data,
        subject_name=farm_id,
        selected_features=features,
        target=target,
        subject_type=subject_type,
        prior_mean=prior_mean,
        prior_cov=prior_cov,
        beta=beta
    )
    result = model.fit_model()

    # Print posterior mean and standard deviation for each feature
    print(f"\nPosterior Mean and Standard Deviation for each feature:\n")
    for feature, stats in result.items():
        print(f"{feature}: mean = {stats['mu']}, std_dev = {stats['sigma']}")

    # Extract the posterior mean for CumulativeHeatLoad
    cumulative_heatload_posterior_mean = result['CumulativeHeatLoad']['mu']

    return cumulative_heatload_posterior_mean

# Train the model with the CumulativeHeatLoad feature on the training set and print results
cumulative_heatload_posterior_mean = fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmCumulativeHeatLoadMilkProduction': cumulative_heatload_posterior_mean
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 1.0007464812391016
Standard Deviation of NormalizedDailyYield: 0.17820337652716353
Variance of NormalizedDailyYield: 0.03175644340568202
Correlation between CumulativeHeatLoad and NormalizedDailyYield: -0.04442987522446997

Selected features: ['CumulativeHeatLoad']

Prior Mean and Standard Deviation for each feature:

Off-set: mean = 1.0007464812391016, std_dev = 0.17820337652716353
CumulativeHeatLoad: mean = -0.04442987522446997, std_dev = 1.0000104462098542

Posterior Mean and Standard Deviation for each feature:

Off-set: mean = 1.000635522373288, std_dev = 0.0008065814288747378
CumulativeHeatLoad: mean = -0.00784026840920556, std_dev = 0.0008065814288747346


,FarmName_Pseudo,FarmCumulativeHeatLoadMilkProduction
0,a624fb9a,-0.00784


In [10]:
# Set subject type to 'farm'
subject_type = 'farm'

# Specify the farm ID for analysis
farm_id = '5c06d92d'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Scale the CumulativeHeatLoad feature
scaler = StandardScaler()
train_data['CumulativeHeatLoad'] = scaler.fit_transform(train_data[['CumulativeHeatLoad']])
val_data['CumulativeHeatLoad'] = scaler.transform(val_data[['CumulativeHeatLoad']])

# Define the single feature
features = ['CumulativeHeatLoad']

# Calculate the correlation between CumulativeHeatLoad and NormalizedDailyYield
cumulative_heatload_correlation = train_data['CumulativeHeatLoad'].corr(train_data[target])
print("Correlation between CumulativeHeatLoad and NormalizedDailyYield:", cumulative_heatload_correlation)

# Function to fit and get the posterior mean for CumulativeHeatLoad
def fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation):
    print(f"\nSelected features: {features}")

    # Calculate prior means for the features
    prior_mean_values = [cumulative_heatload_correlation]  # Use correlation as prior mean for CumulativeHeatLoad
    prior_std_values = [train_data[feature].std() for feature in features]
    
    # Define priors
    prior_mean = np.array([normalized_mean] + prior_mean_values).reshape(-1, 1)
    prior_std = np.array([np.sqrt(normalized_variance)] + prior_std_values).reshape(-1, 1)
    prior_cov = np.eye(len(features) + 1) * 0.01  # Smaller value for stronger prior confidence
    beta = 1 / normalized_variance

    # Print prior means and standard deviations
    print(f"\nPrior Mean and Standard Deviation for each feature:\n")
    for feature, mean, std in zip(['Off-set'] + features, prior_mean.flatten(), prior_std.flatten()):
        print(f"{feature}: mean = {mean}, std_dev = {std}")

    # Initialize and fit the model
    model = BayesianLinearRegression(
        dataframe=train_data,
        subject_name=farm_id,
        selected_features=features,
        target=target,
        subject_type=subject_type,
        prior_mean=prior_mean,
        prior_cov=prior_cov,
        beta=beta
    )
    result = model.fit_model()

    # Print posterior mean and standard deviation for each feature
    print(f"\nPosterior Mean and Standard Deviation for each feature:\n")
    for feature, stats in result.items():
        print(f"{feature}: mean = {stats['mu']}, std_dev = {stats['sigma']}")

    # Extract the posterior mean for CumulativeHeatLoad
    cumulative_heatload_posterior_mean = result['CumulativeHeatLoad']['mu']

    return cumulative_heatload_posterior_mean

# Train the model with the CumulativeHeatLoad feature on the training set and print results
cumulative_heatload_posterior_mean = fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmCumulativeHeatLoadMilkProduction': cumulative_heatload_posterior_mean
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 1.0013067878458104
Standard Deviation of NormalizedDailyYield: 0.13131819898745473
Variance of NormalizedDailyYield: 0.017244469385308756
Correlation between CumulativeHeatLoad and NormalizedDailyYield: -0.020666006338815886

Selected features: ['CumulativeHeatLoad']

Prior Mean and Standard Deviation for each feature:

Off-set: mean = 1.0013067878458104, std_dev = 0.13131819898745473
CumulativeHeatLoad: mean = -0.020666006338815886, std_dev = 1.0000029947068558

Posterior Mean and Standard Deviation for each feature:

Off-set: mean = 1.0011541229780263, std_dev = 0.00032634512513335997
CumulativeHeatLoad: mean = -0.0027557656083748213, std_dev = 0.0003263451251333571


,FarmName_Pseudo,FarmCumulativeHeatLoadMilkProduction
0,a624fb9a,-0.007840
1,5c06d92d,-0.002756


In [11]:
# Set subject type to 'farm'
subject_type = 'farm'

# Specify the farm ID for analysis
farm_id = '752efd72'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Scale the CumulativeHeatLoad feature
scaler = StandardScaler()
train_data['CumulativeHeatLoad'] = scaler.fit_transform(train_data[['CumulativeHeatLoad']])
val_data['CumulativeHeatLoad'] = scaler.transform(val_data[['CumulativeHeatLoad']])

# Define the single feature
features = ['CumulativeHeatLoad']

# Calculate the correlation between CumulativeHeatLoad and NormalizedDailyYield
cumulative_heatload_correlation = train_data['CumulativeHeatLoad'].corr(train_data[target])
print("Correlation between CumulativeHeatLoad and NormalizedDailyYield:", cumulative_heatload_correlation)

# Function to fit and get the posterior mean for CumulativeHeatLoad
def fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation):
    print(f"\nSelected features: {features}")

    # Calculate prior means for the features
    prior_mean_values = [cumulative_heatload_correlation]  # Use correlation as prior mean for CumulativeHeatLoad
    prior_std_values = [train_data[feature].std() for feature in features]
    
    # Define priors
    prior_mean = np.array([normalized_mean] + prior_mean_values).reshape(-1, 1)
    prior_std = np.array([np.sqrt(normalized_variance)] + prior_std_values).reshape(-1, 1)
    prior_cov = np.eye(len(features) + 1) * 0.01  # Smaller value for stronger prior confidence
    beta = 1 / normalized_variance

    # Print prior means and standard deviations
    print(f"\nPrior Mean and Standard Deviation for each feature:\n")
    for feature, mean, std in zip(['Off-set'] + features, prior_mean.flatten(), prior_std.flatten()):
        print(f"{feature}: mean = {mean}, std_dev = {std}")

    # Initialize and fit the model
    model = BayesianLinearRegression(
        dataframe=train_data,
        subject_name=farm_id,
        selected_features=features,
        target=target,
        subject_type=subject_type,
        prior_mean=prior_mean,
        prior_cov=prior_cov,
        beta=beta
    )
    result = model.fit_model()

    # Print posterior mean and standard deviation for each feature
    print(f"\nPosterior Mean and Standard Deviation for each feature:\n")
    for feature, stats in result.items():
        print(f"{feature}: mean = {stats['mu']}, std_dev = {stats['sigma']}")

    # Extract the posterior mean for CumulativeHeatLoad
    cumulative_heatload_posterior_mean = result['CumulativeHeatLoad']['mu']

    return cumulative_heatload_posterior_mean

# Train the model with the CumulativeHeatLoad feature on the training set and print results
cumulative_heatload_posterior_mean = fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmCumulativeHeatLoadMilkProduction': cumulative_heatload_posterior_mean
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 1.0015880766895515
Standard Deviation of NormalizedDailyYield: 0.1030289029037514
Variance of NormalizedDailyYield: 0.010614954833550634
Correlation between CumulativeHeatLoad and NormalizedDailyYield: -0.053897235842989534

Selected features: ['CumulativeHeatLoad']

Prior Mean and Standard Deviation for each feature:

Off-set: mean = 1.0015880766895515, std_dev = 0.1030289029037514
CumulativeHeatLoad: mean = -0.053897235842989534, std_dev = 1.0000038337604782

Posterior Mean and Standard Deviation for each feature:

Off-set: mean = 1.001879418673148, std_dev = 0.0002851307692858588
CumulativeHeatLoad: mean = -0.005549887309382768, std_dev = 0.00028513076928585873


,FarmName_Pseudo,FarmCumulativeHeatLoadMilkProduction
0,a624fb9a,-0.007840
1,5c06d92d,-0.002756
2,752efd72,-0.005550


In [12]:
# Set subject type to 'farm'
subject_type = 'farm'

# Specify the farm ID for analysis
farm_id = 'f454e660'

# Filter data for the specific farm
farm_data = milk_data[milk_data['FarmName_Pseudo'] == farm_id]

# Check if NormalizedDailyYield is centered around 1
normalized_mean = farm_data['NormalizedDailyYield'].mean()
normalized_variance = farm_data['NormalizedDailyYield'].var()
print("Mean of NormalizedDailyYield:", normalized_mean)
print("Standard Deviation of NormalizedDailyYield:", farm_data['NormalizedDailyYield'].std())
print("Variance of NormalizedDailyYield:", normalized_variance)

# Define the target variable
target = 'NormalizedDailyYield'

# Split the data into train and validation sets
train_data, val_data = train_test_split(farm_data, test_size=0.3, random_state=42)

# Scale the CumulativeHeatLoad feature
scaler = StandardScaler()
train_data['CumulativeHeatLoad'] = scaler.fit_transform(train_data[['CumulativeHeatLoad']])
val_data['CumulativeHeatLoad'] = scaler.transform(val_data[['CumulativeHeatLoad']])

# Define the single feature
features = ['CumulativeHeatLoad']

# Calculate the correlation between CumulativeHeatLoad and NormalizedDailyYield
cumulative_heatload_correlation = train_data['CumulativeHeatLoad'].corr(train_data[target])
print("Correlation between CumulativeHeatLoad and NormalizedDailyYield:", cumulative_heatload_correlation)

# Function to fit and get the posterior mean for CumulativeHeatLoad
def fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation):
    print(f"\nSelected features: {features}")

    # Calculate prior means for the features
    prior_mean_values = [cumulative_heatload_correlation]  # Use correlation as prior mean for CumulativeHeatLoad
    prior_std_values = [train_data[feature].std() for feature in features]
    
    # Define priors
    prior_mean = np.array([normalized_mean] + prior_mean_values).reshape(-1, 1)
    prior_std = np.array([np.sqrt(normalized_variance)] + prior_std_values).reshape(-1, 1)
    prior_cov = np.eye(len(features) + 1) * 0.01  # Smaller value for stronger prior confidence
    beta = 1 / normalized_variance

    # Print prior means and standard deviations
    print(f"\nPrior Mean and Standard Deviation for each feature:\n")
    for feature, mean, std in zip(['Off-set'] + features, prior_mean.flatten(), prior_std.flatten()):
        print(f"{feature}: mean = {mean}, std_dev = {std}")

    # Initialize and fit the model
    model = BayesianLinearRegression(
        dataframe=train_data,
        subject_name=farm_id,
        selected_features=features,
        target=target,
        subject_type=subject_type,
        prior_mean=prior_mean,
        prior_cov=prior_cov,
        beta=beta
    )
    result = model.fit_model()

    # Print posterior mean and standard deviation for each feature
    print(f"\nPosterior Mean and Standard Deviation for each feature:\n")
    for feature, stats in result.items():
        print(f"{feature}: mean = {stats['mu']}, std_dev = {stats['sigma']}")

    # Extract the posterior mean for CumulativeHeatLoad
    cumulative_heatload_posterior_mean = result['CumulativeHeatLoad']['mu']

    return cumulative_heatload_posterior_mean

# Train the model with the CumulativeHeatLoad feature on the training set and print results
cumulative_heatload_posterior_mean = fit_and_get_cumulative_heatload_posterior(farm_id, train_data, features, cumulative_heatload_correlation)

# Create a new DataFrame for the current farm's result
new_result = pd.DataFrame([{
    'FarmName_Pseudo': farm_id,
    'FarmCumulativeHeatLoadMilkProduction': cumulative_heatload_posterior_mean
}])

# Check if results_df is empty before concatenation
if results_df.empty:
    results_df = new_result
else:
    results_df = pd.concat([results_df, new_result], ignore_index=True)

results_df

Mean of NormalizedDailyYield: 1.001072448009299
Standard Deviation of NormalizedDailyYield: 0.2353243709243429
Variance of NormalizedDailyYield: 0.05537755955093773
Correlation between CumulativeHeatLoad and NormalizedDailyYield: -0.013619196861415873

Selected features: ['CumulativeHeatLoad']

Prior Mean and Standard Deviation for each feature:

Off-set: mean = 1.001072448009299, std_dev = 0.2353243709243429
CumulativeHeatLoad: mean = -0.013619196861415873, std_dev = 1.0000086202066265

Posterior Mean and Standard Deviation for each feature:

Off-set: mean = 1.000955152305743, std_dev = 0.0009631696193216182
CumulativeHeatLoad: mean = -0.0031592298954186766, std_dev = 0.0009631696193216219


,FarmName_Pseudo,FarmCumulativeHeatLoadMilkProduction
0,a624fb9a,-0.007840
1,5c06d92d,-0.002756
2,752efd72,-0.005550
3,f454e660,-0.003159
